# Lesson 03
# Peter Lorenz

## 0. Preliminaries
First import the required libraries:

In [5]:
import numpy as np
import pandas as pd

Check pandas version:

In [3]:
print("Pandas version: " + pd.__version__)

Pandas version: 1.0.3


Set some global options:

In [6]:
# Display plots inline
%matplotlib inline

# Display multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Suppress scientific notation
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 1. Read Data

Next import the data set:

In [47]:
# Internet location of the data set
url = "https://library.startlearninglabs.uw.edu/DATASCI420/2019/Datasets/SuperstoreTransaction.csv"

# Download the data into a dataframe object
transaction_data = pd.read_csv(url)

# Display shape and initial data
transaction_data.shape
transaction_data.head()

(9994, 21)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,0.000,41.914
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,3,0.000,219.582
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620,2,0.000,6.871
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.577,5,0.450,-383.031
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.200,2.516


Note that there are 9994 transactions in the data set and 21 columns.

## 2. Calculate the Recency, Frequency, Monitory for each customer

In this section, we compute the *recency*, *frequency*, and *monitory* for each customer. First we check the data types of the columns in our data set to determine whether any data type conversions are necessary:

In [48]:
# Display info for each column including data type
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

Our date features appear to be represented in the data set as strings, so we convert them to the datetime data type:

In [49]:
# Convert date columns to datetime data type
transaction_data['Order Date'] = pd.to_datetime(transaction_data['Order Date'],
                                                format = '%m/%d/%Y')
transaction_data['Ship Date'] = pd.to_datetime(transaction_data['Ship Date'],
                                               format = '%m/%d/%Y')

Because the transactions in the data set represent transactions rather than daily spending, we must perform aggregation on the individual transactions to obtain daily spending data per customer. We use the date of the order rather than the ship date as the date on which the transaction occurred:

In [64]:
# Aggregate transaction data per customer per day and reset the index
transaction_agg = transaction_data.groupby(['Customer ID', 
                                            'Order Date'], as_index=False).agg({'Sales': 'sum', 
                                                                                'Quantity': 'sum'})
transaction_agg.reset_index()

# Diagnostic output
transaction_agg.shape
transaction_agg.dtypes

,index,Customer ID,Order Date,Sales,Quantity,Region,Segment
0,0,AA-10315,2014-03-31,726.548,4,West,Consumer
1,1,AA-10315,2014-09-15,29.500,5,East,Consumer
2,2,AA-10315,2015-10-04,26.960,2,West,Consumer
3,3,AA-10315,2016-03-03,4406.072,14,Central,Consumer
4,4,AA-10315,2017-06-29,374.480,5,Central,Consumer
...,...,...,...,...,...,...,...
4987,4987,ZD-21925,2014-08-27,244.760,9,West,Consumer
4988,4988,ZD-21925,2016-04-03,331.080,8,Central,Consumer
4989,4989,ZD-21925,2016-05-05,16.720,5,South,Consumer
4990,4990,ZD-21925,2016-07-08,839.944,7,South,Consumer


(4992, 6)

Customer ID            object
Order Date     datetime64[ns]
Sales                 float64
Quantity                int64
Region                 object
Segment                object
dtype: object

Note that the number of rows in the data set has decreased from 9994 to 4992 as individual transactions have been consolidated into customer daily spending.

Now we compute the rolling features from the aggregated data:

In [56]:
# Compute rolling features
roll_sum = transaction_agg.groupby('Customer ID').rolling(window = '7D',
                                                          on = 'Order Date')['Sales'].sum()
roll_cnt = transaction_agg.groupby('Customer ID').rolling(window = '7D',
                                                          on = 'Order Date')['Quantity'].sum()
roll_min = transaction_agg.groupby('Customer ID')['Order Date'].diff(periods = 1)

Next we combine these rolling features into a new data frame:

In [58]:
# Create new dataframe for rolling features
transaction_roll = pd.concat([roll_sum, roll_cnt], axis = 1)
transaction_roll.columns = ['sales_roll_sum_7D', 'quantity_roll_sum_7D']
transaction_roll = transaction_roll.reset_index()
transaction_roll['last_visit_ndays'] = roll_min.fillna(pd.Timedelta('100 days'))

To view the results, we combine the aggregated data with the rolling data:

In [61]:
# Combine aggregated data with rolling data
transaction_all = transaction_agg.merge(transaction_roll,
                                        how = "inner", 
                                        on = ['Customer ID', 'Order Date'], 
                                        validate = 'one_to_one')
transaction_all.head()

,Customer ID,Order Date,Sales,Quantity,sales_roll_sum_7D,quantity_roll_sum_7D,last_visit_ndays
0,AA-10315,2014-03-31,726.548,4,726.548,4.000,100 days
1,AA-10315,2014-09-15,29.500,5,29.500,5.000,168 days
2,AA-10315,2015-10-04,26.960,2,26.960,2.000,384 days
3,AA-10315,2016-03-03,4406.072,14,4406.072,14.000,151 days
4,AA-10315,2017-06-29,374.480,5,374.480,5.000,483 days


In the above output, *recency* is found in the *last_visit_ndays* column, *frequency* is found in the *quantity_roll_sum_7D* column, and *monetary* is found in the *sales_roll_sum_7D* column.

## 3. Using K-means algorithm, perform customer segmentation

In this section, we perform customer segmentation using the k-means algorithm. First we convert the timedelta data type of *last_visit_ndays* to integer so we can aggregate it:

In [77]:
transaction_all['last_visit_ndays_int'] = transaction_all['last_visit_ndays'].dt.days

Now we aggregate the RFM data computed in the previous section by customer. For *sales_roll_sum_7D* and *quantity_roll_sum_7D* we aggregate by the median to prevent large orders from unduly influencing the typical value. For *last_visit_ndays_int* we aggregate by the mean.

In [78]:
# Aggregate RFM data per customer and reset the index
rfm_agg = transaction_all.groupby(['Customer ID'],
                                  as_index=False).agg({'sales_roll_sum_7D': 'median',
                                                       'quantity_roll_sum_7D': 'median',
                                                       'last_visit_ndays_int': 'mean'})
rfm_agg.reset_index()

# Diagnostic output
rfm_agg.shape

,index,Customer ID,sales_roll_sum_7D,quantity_roll_sum_7D,last_visit_ndays_int
0,0,AA-10315,374.480,5.000,257.200
1,1,AA-10375,56.860,5.000,158.889
2,2,AA-10480,308.490,9.000,294.250
3,3,AA-10645,717.165,11.500,222.000
4,4,AB-10015,309.592,4.000,365.333
...,...,...,...,...,...
788,788,XP-21865,186.540,8.000,136.091
789,789,YC-21895,750.680,5.000,247.000
790,790,YS-21880,457.500,6.000,146.750
791,791,ZC-21910,148.884,7.000,93.846


(793, 4)

Customer ID              object
sales_roll_sum_7D       float64
quantity_roll_sum_7D    float64
last_visit_ndays_int    float64
dtype: object

After aggregating the RFM data set per customer, we now have 793 rows, one per customer with RFM values aggregated for median *sales_roll_sum_7D*, median *quantity_roll_sum_7D*, and mean *last_visit_ndays_int*.

## 4. Describe your results

TODO